## Library Import

In [1]:
import duckdb
import polars as p

import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from config import load_setting
settings = load_setting()

## Load Silver Data

In [2]:
# Test connection to Silver Data db
try:
    db_path = settings['process']['silver_db_path']
    print(f"Attempting to connect to: {db_path}")
    
    # Try connection
    con = duckdb.connect(f"../{db_path}")
    
    # Test Connection with a simple query
    result = con.execute("SELECT 1").fetchone()
    print("Connection Succesfful!")
    print(f"Test query result: {result}")
    
    
    # List all tables in the database
    tables = con.execute("SHOW TABLES").fetchall()
    print("\nAvailable tables:", tables)
    
except Exception as e:
    print(f"Connection Failed {str(e)}")

Attempting to connect to: process/storage/silver/resampled.duckdb
Connection Succesfful!
Test query result: (1,)

Available tables: [('gold_data',), ('raw_data',), ('silver_1',), ('silver_13',), ('silver_21',), ('silver_3',), ('silver_34',), ('silver_5',), ('silver_55',), ('silver_8',)]


In [3]:
# List all tables
tables = con.execute("SHOW TABLES").fetchall()
print("Available tables:", tables)

Available tables: [('gold_data',), ('raw_data',), ('silver_1',), ('silver_13',), ('silver_21',), ('silver_3',), ('silver_34',), ('silver_5',), ('silver_55',), ('silver_8',)]


In [4]:
# Get schema information for a specific table
# Replace 'your_table_name' with an actual table name
table_info = con.execute("DESCRIBE silver_1").fetchall()
print("Table schema:", table_info)

Table schema: [('symbol', 'VARCHAR', 'YES', None, None, None), ('date', 'TIMESTAMP WITH TIME ZONE', 'YES', None, None, None), ('open', 'DOUBLE', 'YES', None, None, None), ('high', 'DOUBLE', 'YES', None, None, None), ('low', 'DOUBLE', 'YES', None, None, None), ('close', 'DOUBLE', 'YES', None, None, None), ('volume', 'DOUBLE', 'YES', None, None, None)]


In [5]:
# Get sample data from the table
sample_data = con.execute("SELECT * FROM silver_1 LIMIT 5").fetchall()
print("Sample data:", sample_data)

Sample data: [('VRNS', datetime.datetime(2019, 1, 25, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 18.0833320618, 18.4699993134, 17.9133338928, 18.3733329773, 476700.0), ('VRNS', datetime.datetime(2019, 3, 5, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 19.6033325195, 20.2233333588, 19.4233322144, 20.0766677856, 1911600.0), ('VRNS', datetime.datetime(2019, 8, 8, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 22.8866672516, 23.703332901, 22.8866672516, 23.3566665649, 895200.0), ('VRNS', datetime.datetime(2019, 10, 29, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 23.4333324432, 24.6996669769, 22.3366661072, 23.8233337402, 4088700.0), ('VRNS', datetime.datetime(2019, 11, 12, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 24.3366661072, 24.7833328247, 24.2466678619, 24.6700000763, 741900.0)]


In [6]:
# Get the date range of the silver_1 table
con.execute("SELECT MIN(date), MAX(date) FROM silver_1").fetchall() 

[(datetime.datetime(1962, 1, 2, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>),
  datetime.datetime(2025, 7, 10, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>))]

In [7]:
# Get the first 5 rows of Resampled Data
first_5_rows = con.execute("SELECT * FROM silver_1 ORDER BY date DESC LIMIT 5").fetchall()
print("First 5 rows:", first_5_rows)

First 5 rows: [('PPBT', datetime.datetime(2025, 7, 10, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 2.4, 2.58, 2.3501, 2.52, 13071.0), ('VCIG', datetime.datetime(2025, 7, 10, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 1.41, 1.45, 1.28, 1.2937, 923686.0), ('PKBK', datetime.datetime(2025, 7, 10, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 20.815, 21.5, 20.815, 21.25, 7498.0), ('DOGZ', datetime.datetime(2025, 7, 10, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 9.45, 9.67, 9.12, 9.35, 124958.0), ('ANGH', datetime.datetime(2025, 7, 10, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 0.481, 0.498, 0.48, 0.494, 21844.0)]


In [8]:
# Get total row count
row_count = con.execute("SELECT COUNT(*) FROM silver_1").fetchone()
print("Total rows:", row_count)

Total rows: (6210030,)


In [9]:
# Get the first 5 rows of the table
first_5_rows = con.execute("SELECT * FROM silver_1 LIMIT 5").fetchall()
print("First 5 rows:", first_5_rows)
# Get the last 5 rows of the table
last_5_rows = con.execute("SELECT * FROM silver_1 ORDER BY date DESC LIMIT 5").fetchall()
print("Last 5 rows:", last_5_rows)


First 5 rows: [('VRNS', datetime.datetime(2019, 1, 25, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 18.0833320618, 18.4699993134, 17.9133338928, 18.3733329773, 476700.0), ('VRNS', datetime.datetime(2019, 3, 5, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 19.6033325195, 20.2233333588, 19.4233322144, 20.0766677856, 1911600.0), ('VRNS', datetime.datetime(2019, 8, 8, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 22.8866672516, 23.703332901, 22.8866672516, 23.3566665649, 895200.0), ('VRNS', datetime.datetime(2019, 10, 29, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 23.4333324432, 24.6996669769, 22.3366661072, 23.8233337402, 4088700.0), ('VRNS', datetime.datetime(2019, 11, 12, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 24.3366661072, 24.7833328247, 24.2466678619, 24.6700000763, 741900.0)]
Last 5 rows: [('PPBT', datetime.datetime(2025, 7, 10, 0, 0, tzinfo=<DstTzInfo '

In [10]:
# Get column statistics
stats = con.execute("""
    SELECT 
        symbol,
        COUNT(*) as count,
        AVG(open) as avg,
        MIN(open) as min,
        MAX(open) as max
    FROM silver_1
    GROUP BY symbol
""").fetchall()
print("Column statistics:", stats)

Column statistics: [('JOE', 8889, 24.629214031821515, 0.0, 79.3398809016), ('MGEE', 11422, 20.414928078657137, 0.0, 107.1535726834), ('FLWS', 6524, 9.390552573844298, 0.9499999881, 39.6100006104), ('HBI', 4740, 10.705365788086727, 1.0939039001, 26.5186648972), ('BN', 10463, 10.086634492096978, 0.2395896013, 64.25), ('SCYX', 2814, 22.05416125860784, 0.68, 144.1999969482), ('PLXS', 9933, 31.676915422083518, 0.0, 170.3800048828), ('SF', 10578, 16.948359428197804, 0.4213503599, 118.4845054196), ('TFIN', 2682, 47.05687995981794, 12.1499996185, 135.0800018311), ('KBR', 4689, 27.653856985995542, 8.2977626698, 71.5790658811), ('PINS', 1565, 34.858295069629015, 10.6000003815, 87.4100036621), ('SATS', 4408, 28.08404990048219, 9.8800001144, 50.5753631592), ('AB', 9380, 10.36042096407807, 0.0, 42.0039016723), ('HIVE', 3630, 2.972287877186641, 0.1000000015, 26.3500003815), ('DIST', 538, 10.802853119724535, 10.1499996185, 15.94), ('PLAB', 9658, 12.197120389099444, 0.0, 45.5625), ('TAIT', 7607, 1.449

In [11]:
import polars as pl

# Turn stats into a pl dataframe with mixed types allowed
stats_df = pl.DataFrame(stats, 
                        schema={
                            "symbol": pl.Utf8,
                            "count": pl.Int64,
                            "avg": pl.Float64,
                            "min": pl.Float64,
                            "max": pl.Float64
                        })
print(stats_df)

shape: (1_458, 5)
┌────────┬───────┬───────────┬──────────┬────────────┐
│ symbol ┆ count ┆ avg       ┆ min      ┆ max        │
│ ---    ┆ ---   ┆ ---       ┆ ---      ┆ ---        │
│ str    ┆ i64   ┆ f64       ┆ f64      ┆ f64        │
╞════════╪═══════╪═══════════╪══════════╪════════════╡
│ JOE    ┆ 8889  ┆ 24.629214 ┆ 0.0      ┆ 79.339881  │
│ MGEE   ┆ 11422 ┆ 20.414928 ┆ 0.0      ┆ 107.153573 │
│ FLWS   ┆ 6524  ┆ 9.390553  ┆ 0.95     ┆ 39.610001  │
│ HBI    ┆ 4740  ┆ 10.705366 ┆ 1.093904 ┆ 26.518665  │
│ BN     ┆ 10463 ┆ 10.086634 ┆ 0.23959  ┆ 64.25      │
│ …      ┆ …     ┆ …         ┆ …        ┆ …          │
│ SEED   ┆ 5355  ┆ 35.026178 ┆ 0.9574   ┆ 175.199997 │
│ VICI   ┆ 1890  ┆ 22.888159 ┆ 8.341741 ┆ 33.38      │
│ NSIT   ┆ 7666  ┆ 38.486079 ┆ 1.777778 ┆ 225.949997 │
│ MBINL  ┆ 158   ┆ 24.317865 ┆ 23.0371  ┆ 25.215     │
│ SDM    ┆ 47    ┆ 7.286798  ┆ 4.15     ┆ 12.47      │
└────────┴───────┴───────────┴──────────┴────────────┘


/opt/homebrew/Cellar/python@3.10/3.10.17/Frameworks/Python.framework/Versions/3.10/lib/python3.10/functools.py:889: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


In [12]:
con.close()
